In [9]:
import pandas as pd

file_path = r"C:\Users\James sim\Desktop\workspace\미드 프로젝트\인구이동_연도_시군구.csv" # 실제 파일 경로로 수정해주세요.
df = pd.read_csv(file_path, encoding='utf-8')

# 원본 데이터프레임의 모든 연도 컬럼 이름 가져오기
year_columns = [col for col in df.columns if '년' in col]

# 1. 모든 고유 시군구를 원본 df에서 추출
all_sigungus_from_original_df = df['시군구'].unique()
# 2. 모든 고유 연도를 연도 컬럼명에서 추출 (정수형으로 변환)
all_years_from_columns = sorted(list(set([int(col.replace('년', '')) for col in year_columns])))

# 데이터프레임 melt
df_melted = df.melt(
    id_vars=['시도', '시군구', '항목'],
    value_vars=year_columns,
    var_name='연도',
    value_name='이동자수'
)

# '연도' 컬럼에서 숫자만 추출하여 정수형으로 변환
df_melted['연도'] = df_melted['연도'].str.extract(r'(\d+)').astype(int)

# '총전입[명]', '총전출[명]' 항목만 필터링
df_melted_filtered = df_melted[df_melted['항목'].isin(['총전입[명]', '총전출[명]'])]

# 모든 시군구/연도 조합 만들기 (원본에서 추출한 목록 사용)
full_index = pd.MultiIndex.from_product(
    [all_sigungus_from_original_df, all_years_from_columns],
    names=['시군구', '연도']
)

# 피벗 테이블 생성 (필터링된 df_melted_filtered 사용)
df_pivot = df_melted_filtered.pivot_table(
    index=['시군구', '연도'],
    columns='항목',
    values='이동자수',
    fill_value=0  # pivot 시 없는 값은 0으로 채움
)

# reindex로 모든 조합 채우기 (fill_value=0은 reindex 시 새로 추가되는 행의 값)
df_pivot = df_pivot.reindex(full_index, fill_value=0).reset_index()

# 컬럼명 변경
df_pivot = df_pivot.rename(columns={'총전입[명]': '총전입', '총전출[명]': '총전출'})

# 이동량 및 이동률 계산 (수정된 부분)
df_pivot['총이동량'] = df_pivot['총전입'] + df_pivot['총전출']
df_pivot['이동률'] = (df_pivot['총이동량'] / 10000).round(1) # F3 =ROUND((F2/10000),1) 적용

# 결과 선택 및 정렬
result = df_pivot[['시군구', '연도', '이동률']].sort_values(['시군구', '연도'])

# CSV 파일로 저장
output_file_path = "시군구별_이동률_2013-2023_revised_formula.csv" # 저장할 파일명
result.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# 결과 확인
print(f"Result saved to {output_file_path}")
print("\nResult head(20):")
print(result.head(20))
print(f"\nNumber of unique SiGunGus in result: {result['시군구'].nunique()}")
print(f"Number of unique SiGunGus in original df: {len(all_sigungus_from_original_df)}")

Result saved to 시군구별_이동률_2013-2023_revised_formula.csv

Result head(20):
항목    시군구    연도   이동률
1140  가평군  2013   1.5
1141  가평군  2014   1.5
1142  가평군  2015   1.5
1143  가평군  2016   1.4
1144  가평군  2017   1.4
1145  가평군  2018   1.5
1146  가평군  2019   1.4
1147  가평군  2020   1.4
1148  가평군  2021   1.4
1149  가평군  2022   1.3
1150  가평군  2023   1.3
1151  가평군  2024   1.3
276   강남구  2013  19.6
277   강남구  2014  22.0
278   강남구  2015  22.3
279   강남구  2016  20.4
280   강남구  2017  19.4
281   강남구  2018  19.8
282   강남구  2019  18.9
283   강남구  2020  19.7

Number of unique SiGunGus in result: 234
Number of unique SiGunGus in original df: 234
